<a href="https://colab.research.google.com/github/jnqeras/ARC/blob/master/tesis4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descripción

En esta notebook voy a hacer lo mimso que en tesis2.ipynb, pero generando los embeddings pasándole una lista de strings a model.encode(), como lo indica la documentación de model.encode().
Lo voy a hacer así, porque si lo hago así, model.encode() devuelve un tensor con los embbedings, que es literalmente lo que toma la función util.tensor().

Si generase los embeddings con un for loop, como lo hacía en tesis3.ipynb, después tengo que transofrmar la lista de embeddings en un tensor y las dimensiones quedan raras y no se le pueden pasar a util.cos_sim.

No voy a chequear si los embeddings generados de a uno son los mismos que los generados con la lista, voy a asumir que esto es así.

# **Set-up**



Installation
We recommend Python 3.6 or higher, PyTorch 1.6.0 or higher and transformers v4.6.0 or higher. The code does not work with Python 2.7.

Install SentenceTransformers

New section
Install with pip

Install the sentence-transformers with pip:

In [1]:
# Fixme: Para correr el script localmente, comentar esta línea (para correr en Colab, descomentar):
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 5.5 MB 38.8 MB/s 
     |████████████████████████████████| 1.3 MB 51.3 MB/s 
     |████████████████████████████████| 182 kB 47.3 MB/s 
     |████████████████████████████████| 7.6 MB 23.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=04cd703faa589c77c2674af3baef18e162ea60e674ffc55cdc6039589be09013
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
import random
import torch
import pickle # to store and load the embeddings
import sys
import time
import csv



device = torch.device("cuda:1")

# **Trabajando con el Dataset de CONAN:**


In [3]:
import pandas as pd     #Fixme: cuando esté todo terminado, puedo mover todos los imports a la parte superior.
import numpy as np

url = 'https://raw.githubusercontent.com/jnqeras/CONANDatasetCopy/main/CONAN/CONAN.csv'

df1 = pd.read_csv(url)

# Dataset is now stored in a Pandas Dataframe

In [4]:
# Observo los primeros cinco elementos del Dataframe.
df1.head()

,cn_id,hateSpeech,counterSpeech,hsType,hsSubType,cnType,age,gender,educationLevel
0,ENT1ST0001HS0033CN000021,"According to a recent OFSTED report, a school ...","To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
1,ENT1ST0001HS0033CN000021P1,In Birmingham there is a school where girls an...,"To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
2,ENT1ST0001HS0033CN000021P2,A school in Birningham is still separating gir...,"To be fair, the OFSTED report is more concerne...",Islamophobia,crimes,facts,61.0,male,Bachelor
3,ENT1ST0001HS0033CN000022,"According to a recent OFSTED report, a school ...",While this story is presented as an 'Islamic i...,Islamophobia,crimes,facts,61.0,male,Bachelor
4,ENT1ST0001HS0033CN000022P1,In Birmingham there is a school where girls an...,While this story is presented as an 'Islamic i...,Islamophobia,crimes,facts,61.0,male,Bachelor


In [5]:
# Selecciono las dos columnas que me interesan
sentences_conan = df1[['hateSpeech', 'counterSpeech']]

sentences_conan[0:1000]

,hateSpeech,counterSpeech
0,"According to a recent OFSTED report, a school ...","To be fair, the OFSTED report is more concerne..."
1,In Birmingham there is a school where girls an...,"To be fair, the OFSTED report is more concerne..."
2,A school in Birningham is still separating gir...,"To be fair, the OFSTED report is more concerne..."
3,"According to a recent OFSTED report, a school ...",While this story is presented as an 'Islamic i...
4,In Birmingham there is a school where girls an...,While this story is presented as an 'Islamic i...
...,...,...
995,Muslims do not have nothing useful that can en...,Muslims have been part of the British fabric s...
996,Muslims have nothing to contribute to our soci...,Do you know that you are using a lot of things...
997,Muslims cannot add any value to our culture.,Do you know that you are using a lot of things...
998,Muslims do not have nothing useful that can en...,Do you know that you are using a lot of things...


Hago una selección de únicamemente las contranarrativas (para armar el ránking que consiste en contranarrativas exclusivamente).

In [6]:
# Selecciono la columna de contranarrativas
counternarratives_conan = df1[['counterSpeech']]

counternarratives_conan[0:1000]

,counterSpeech
0,"To be fair, the OFSTED report is more concerne..."
1,"To be fair, the OFSTED report is more concerne..."
2,"To be fair, the OFSTED report is more concerne..."
3,While this story is presented as an 'Islamic i...
4,While this story is presented as an 'Islamic i...
...,...
995,Muslims have been part of the British fabric s...
996,Do you know that you are using a lot of things...
997,Do you know that you are using a lot of things...
998,Do you know that you are using a lot of things...


In [7]:
#sólo para hacer un sanity check, voy a seleccionar los discursos de odio de CONAN, eliminar los repetidos 
# y aseurarme que se cumple sentences_conan_list_sin_repetidos = counternarratives_conan_list_sin_repetidos + hate_speech_conan_list_sin_repetidos
# Selecciono las dos columnas que me interesan
hate_speech_conan = df1[['hateSpeech']]

hate_speech_conan [0:1000]

,hateSpeech
0,"According to a recent OFSTED report, a school ..."
1,In Birmingham there is a school where girls an...
2,A school in Birningham is still separating gir...
3,"According to a recent OFSTED report, a school ..."
4,In Birmingham there is a school where girls an...
...,...
995,Muslims do not have nothing useful that can en...
996,Muslims have nothing to contribute to our soci...
997,Muslims cannot add any value to our culture.
998,Muslims do not have nothing useful that can en...


Creo embeddings para los mensajes de odio y sus contranarrativas.

In [8]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')         
#fixme: acá está la documentación de este modelo, si cambio de modelo, cambiará el formato en el que tengo que pasarle las frases que quiero encodear (tuits odio y contranarrativas).

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

*Convierto* los dataframes en dos listas de strings (porque es lo que toma el modelo 'all-MiniLM-L6-v2'). Voy a eliminar los repetidos.


In [9]:
def removeDuplicatesFromList(list):
  # Removes duplicated from list
  # using list comprehension + enumerate()
  
  res = [i for n, i in enumerate(list) if i not in list[:n]]
  return res

In [10]:
# convierto el dataframe en una lista sin repetidos
sentences_conan_list_sin_repetidos = removeDuplicatesFromList(list(sentences_conan.values.flatten()));

In [11]:
counternarratives_conan_list_sin_repetidos = removeDuplicatesFromList(list(counternarratives_conan.values.flatten()));


In [12]:
hate_speech_conan_list_sin_repetidos = removeDuplicatesFromList(list(hate_speech_conan.values.flatten()));


Fixme: acá podría chequear si sentences_conan_list_sin_repetidos y counternarratives_conan_list_sin_repetidos efectivamente no tienen repetidos.

In [13]:
#Veo cuántos elementos tiene sentences_conan_list eliminado repetidos.
len(sentences_conan_list_sin_repetidos)

7659

In [14]:
#Veo cuántos elementos tiene counternarratives_conan_list eliminado repetidos.
len(counternarratives_conan_list_sin_repetidos)

6803

In [15]:
#Veo cuántos elementos tiene hate_speech_conan_list_sin_repetidos eliminado repetidos.
len(hate_speech_conan_list_sin_repetidos)

856

In [16]:
# Chequeo si da la suma:
len(sentences_conan_list_sin_repetidos) == len(counternarratives_conan_list_sin_repetidos) + len(hate_speech_conan_list_sin_repetidos)

True

#Genero los embeddings de los discursos de odio a partir de la lista de discursos de odio sin repetidos.

In [17]:
print('Generando Embeddings para los discursos de odio.')

Generando Embeddings para los discursos de odio.


In [18]:
# Fixme: Comentar la siguiente celda si los embeddings ya están generados.
# Genero los embeddings y los guardo en el disco. 


# Genero los embeddings
embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista = model.encode(hate_speech_conan_list_sin_repetidos, convert_to_tensor=True);

# Store embeddings on disc
with open('embeddings_hate_speech.pkl', "wb") as fOut:
    pickle.dump({'embeddings': embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista}, fOut, protocol=pickle.HIGHEST_PROTOCOL)


In [19]:
# Fixme: comentar la siguiente celda genero los embeddings por lo que no hace falta cargarlos.
# Load embeddings from disc
with open('embeddings_hate_speech.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista = stored_data['embeddings']

## La siguiente celda debe utilizarse cuando haya GPUs disponibles.

In [20]:
# Envío los embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista a la GPU:
# embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista.to(device)

Busco la dimensión de embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista

In [21]:
len(embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista)

856

In [22]:
len(embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[3])

384

embeddings_hate_speech_conan_list_sin_repetidosr_creado_por_lista es una matriz de 856 x 384 = cantidad_mensajes_de_odio_sin_repetir x largo_de_cada_embedding

#Genero los embeddings para las contranarrativas a partir de la lista de contranarrativas sin repetidos.

In [23]:
print('Generando Embeddings para las contranarrativas.')

Generando Embeddings para las contranarrativas.


In [24]:
# Fixme: Comentar la siguiente celda si los embeddings ya están generados.
# Genero los embeddings y los guardo en el disco. 


# Genero los embeddings
# If available, the model is automatically executed on the GPU.
embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista = model.encode(counternarratives_conan_list_sin_repetidos, convert_to_tensor=True);

# Store embeddings on disc
with open('embeddings_counternarratives.pkl', "wb") as fOut:
    pickle.dump({'embeddings': embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista}, fOut, protocol=pickle.HIGHEST_PROTOCOL)


In [25]:
# Fixme: comentar la siguiente celda genero los embeddings por lo que no hace falta cargarlos.
# Load embeddings from disc
with open('embeddings_counternarratives.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista = stored_data['embeddings']

## La siguiente celda debe utilizarse cuando haya GPUs disponibles.

In [26]:
# Envío los embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista a la GPU:
# embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista.to(device)

Busco la dimensión de embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista

In [27]:
len(embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)

6803

In [28]:
len(embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[0])

384

embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista es una matriz de 6803 x 384 = cantidad_contranarrativas_sin_repetir x largo_de_cada_embedding.

# **Ploteo de Embeddings.**

Fixme: la sugerencia de Lau y Dami es graficar usando PCA o T-SNE

Intentar plotear como en esta página: https://inside-machinelearning.com/en/efficient-sentences-embedding-visualization-tsne/

Este ploteo está bueno: https://github.com/ashutoshsingh25/Plotting-multidimensional-vectors-using-t-SNE/blob/master/TSNE%20Code%20for%20clusring%20image%20and%20text%20vectors%20with%20labels.ipynb

Este ploteo está bueno: https://towardsdatascience.com/plotting-text-and-image-vectors-using-t-sne-d0e43e55d89

k-Means: para encontrar el valor óptimo de k ver Gap Statistic: https://hastie.su.domains/Papers/gap.pdf

Si no, chusmear: Cluster: https://towardsdatascience.com/plotting-text-and-image-vectors-using-t-sne-d0e43e55d89
    


#**Prueba de hipótesis a mano: un embedding calculado como propone la hipótesis comparado contra todo el dataset**

Voy a hacer una prueba a mano de la versión del experimento que es más fácil de implementar, es decir:
1. Partir del embedding de contranarrativa_1.
2. Al embedding de contranarrativa_1, restarle el embedding de odio_1. Suponiendo que esto nos deja en el espacio vectorial cercano a "contranarrativa sin mensaje de odio asignado".
3. Sumarle el embedding de odio_2 y esperar que esto nos deje en el espacio vectorial cercano al embedding de contranarrativa_2.



El plan es:

1. Voy a elegir una contranarrativa_1 a mano y voy a buscar su sentence embedding.
2. Voy a elegir odio_1 (el mensaje de odio que le corresponde a contranarrativa_1 en el dataset del CONAN), voy a buscar su sentence embedding y restarselo a embedding de contranarrativa_1.
3. Voy a elegir un mensaje odio_2, voy a buscar su sentence embedding y voy a sumárselo al resultado del paso 2.
4. Voy a calcular la similaridad coseno entre el resultado del paso anterior y los embeddings de todas las **contranarratiavs del Conan**. Espero que la que tenga valor más alto, corresponda a una contranarrativa que "sirva" (en sentido de que contraargumente) contra al mensaje odio_2, idealmente quiero que sea una contranarrativa que en el dataset original (Conan), aparezca asignada a odio_2. 

## **Nota al márgen: Cómo elegir pares de contranarrativas y discursos de odio para las pruebas a mano**

Se puede elegir un odio_2 tal que sea muy diferente a odio_1 (también se puede elegir un odio_2 tal que sea muy similar a odio_1), además hay discursos de odio a los que se responde con distintas contranarrativas, puedo elegir odio_1=odio_2 y contranarrativa_1 != contranarrativa_2. También los puedo elegir según su similitud coseno (similitud coseno entre contranarrativa_1 y contranarrativa_2 y entre odio_1 y odio_2).

## **Prueba a mano n1:**

En esta primera prueba, voy a elegir los mensaje de odio y sus contranarrativas sin pensar demasiado cómo los elijo.

Elijo:

* contranarrativa_1: To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.

El mensaje de odio que le corresponde es:
* odio_1: 	According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.


* contranarrativa_2: While this story is presented as an 'Islamic issue', the OFSTED report refers to 'countless' faith schools still that still illegally maintain gender segregation - my guess is this is a cross-faith practice and not exclusive to Islamic faith schools.

El mensaje de odio que le corresponde es:

* odio_2: In Birmingham there is a school where girls and boys are separeted even if it isn't legal.	


#### **Ejecución del plan:**

##### 1. Voy a elegir una contranarrativa_1 a mano y voy a buscar su sentence embedding

* contranarrativa_1: To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.

In [29]:
#Busco el índice de contranarrativa_1.
i = counternarratives_conan_list_sin_repetidos.index('To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.')

In [30]:
#Chequeo que efectivamente el índice obtenido corresponda a contranarrativa_1 (si es así, esta celda debe devolver true):
counternarratives_conan_list_sin_repetidos[i] == 'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.'

True

In [31]:
contranarrativa_1_embedding_creado_por_lista = embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[i];

In [32]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_contranarrativa_1
#Genero el embedding para la contranarrativa_1 
#embedding_contranarrativa_1 = model.encode(counternarratives_conan_list_sin_repetidos[i], convert_to_tensor=True)

In [33]:
# Fixme: yo espero que esta comparación me de true, pero da false. 
#(contranarrativa_1_embedding_creado_por_lista==embedding_contranarrativa_1).all()

In [34]:
#contranarrativa_1_embedding_creado_por_lista.shape

In [35]:
#embedding_contranarrativa_1.shape

In [36]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_contranarrativa_1

#Chequeo si el embedding_contranarrativa_1 es igual al embedding que cree con la lista más arriba (esta 
# celda no hace al experimento todavía, pero me sirve para saber que puedo calcular los embeddings de varias
# maneras). Si los embeddings son iguales, esta celda debe devolver true:
#np.array_equal(embedding_contranarrativa_1[0], embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[i])

##### 2. Voy a elegir odio_1 (el mensaje de odio que le corresponde a contranarrativa_1 en el dataset del CONAN), voy a generar su sentence embedding y restarselo a embedding de contranarrativa_1.



Se que odio_1 es el siguiente, porque lo encuentro revisando el dataframe del conan:

* odio_1: According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.

In [37]:
# Busco el índice de odio_1.
j = hate_speech_conan_list_sin_repetidos.index('According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.')

In [38]:
#Chequeo que efectivamente el índice obtenido corresponda a odio_1  (si es así, esta celda debe devolver true):
hate_speech_conan_list_sin_repetidos[j] == 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.'

True

In [39]:
odio_1_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[j];

In [40]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_1

#Genero el embedding para odio_1
#embedding_odio_1 = model.encode(hate_speech_conan_list_sin_repetidos[j], convert_to_tensor=True)

In [41]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_1

#Chequeo si el embedding_odio_1 es igual al embedding que cree con un for loop más arriba (esta celda no
# hace al experimento todavía, pero me sirve para saber que puedo calcular los embeddings de varias maneras):
#np.array_equal(embedding_odio_1, embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[j])

In [42]:
# Resto el embedding de odio_1 al de contranarrativa_1.
embedding_contranarrativa_1_sin_discurso_de_odio_1 = contranarrativa_1_embedding_creado_por_lista - odio_1_embedding_creado_por_lista

In [43]:
# Sanity check: comparo con la función subtract (hace element-wise substraction). 
# Espero que esta celda me de true.
embedding_contranarrativa_1_sin_discurso_de_odio_1_subtract = np.subtract(contranarrativa_1_embedding_creado_por_lista, odio_1_embedding_creado_por_lista)

(embedding_contranarrativa_1_sin_discurso_de_odio_1_subtract == embedding_contranarrativa_1_sin_discurso_de_odio_1).all()

tensor(True)

##### 3. Voy a elegir un mensaje odio_2, voy a buscar su sentence embedding y voy a sumárselo al resultado del paso 2.

* odio_2: In Birmingham there is a school where girls and boys are separeted even if it isn't legal.

In [44]:
# Busco el índice de odio_2 para obtener su sentence embedding.
k = hate_speech_conan_list_sin_repetidos.index('In Birmingham there is a school where girls and boys are separeted even if it isn\'t legal.')

In [45]:
#Chequeo que efectivamente el índice obtenido corresponda a odio_2  (si es así, esta celda debe devolver true):
hate_speech_conan_list_sin_repetidos[k] == 'In Birmingham there is a school where girls and boys are separeted even if it isn\'t legal.'

True

In [46]:
odio_2_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[k];

In [47]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_2

#Genero el embedding para odio_2
#embedding_odio_2 = model.encode([hate_speech_conan_list_sin_repetidos[k]], convert_to_tensor=True)

In [48]:
# Comento esta celda, porque como no está dando el mismo embedding, no quiero por error utilizar embedding_odio_2

#Chequeo si el embedding_odio_2 es igual al embedding que cree con un for loop más arriba (esta celda no
# hace al experimento todavía, pero me sirve para saber que puedo calcular los embeddings de varias maneras):
#np.array_equal(embedding_odio_2, embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[k])

In [49]:
# Sumo el embedding de odio_2 al resultado del paso 2 (embedding_contranarrativa_1_sin_discurso_de_odio_1).
embedding_cercano_a_contranarrativa_para_odio_2 = embedding_contranarrativa_1_sin_discurso_de_odio_1 + odio_2_embedding_creado_por_lista

In [50]:
# chequeo que el embedding embedding_cercano_a_contranarrativa_para_odio_2 tenga el mismo shape que los
# embeddings de embeddings_hate_speech_conan_list_sin_repetidos_creado_con_for y que el emmeding
# embedding_contranarrativa_1
embedding_cercano_a_contranarrativa_para_odio_2.shape == embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[0].shape == contranarrativa_1_embedding_creado_por_lista.shape

True

In [51]:
# Sanity check: comparo con la función add (hace element-wise addition). 
# Espero que esta celda me de true.
embedding_cercano_a_contranarrativa_para_odio_2_add = np.add(embedding_contranarrativa_1_sin_discurso_de_odio_1, odio_2_embedding_creado_por_lista)

(embedding_cercano_a_contranarrativa_para_odio_2_add == embedding_cercano_a_contranarrativa_para_odio_2).all()

tensor(True)

##### 4. Voy a calcular la similaridad coseno entre el resultado del paso anterior (embedding_cercano_a_contranarrativa_para_odio_2) y los embeddings de todas las contranarrativas del Conan (embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista). 

Espero que la que tenga valor más alto, corresponda a una contranarrativa que "sirva" (en sentido de que contraargumente) contra el mensaje odio_2, idealmente quiero que sea una contranarrativa que en el dataset original (Conan), aparezca asignada a odio_2.

La contranarrativa correspondiente a odio_2 en el dataset es: 
contranarrativa_2: While this story is presented as an 'Islamic issue', the OFSTED report refers to 'countless' faith schools still that still illegally maintain gender segregation - my guess is this is a cross-faith practice and not exclusive to Islamic faith schools.

Como mejor caso, quisiera que el embedding con similitud coseno más grande con embedding_cercano_a_contranarrativa_para_odio_2, sea el embedding de contranarrativa_2.

Antes de calcular la similaridad coseno entre embedding_cercano_a_contranarrativa_para_odio_2 y embeddings_counternarratives_conan_list_sin_repetidos_creado_con_for, voy a examinarlos para estar seguro de que son comparables.

Puedo ver que uno es una lista embeddings y el otro es un embedding.

In [52]:
# Computo la similaridad coseno de embedding_cercano_a_contranarrativa_para_odio_2, 
#contra todos los embeddings de todas las contranarrativas del Conan.

# Calculo la similaridad coseno 
cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan = util.cos_sim(embedding_cercano_a_contranarrativa_para_odio_2, embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)

In [53]:
cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan.shape 

torch.Size([1, 6803])

In [54]:
len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0])

6803

In [55]:
cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan

tensor([[0.8228, 0.4275, 0.3856,  ..., 0.2685, 0.0595, 0.1089]])

In [56]:
# Chequeo que tengo una distancia coseno por cada elemento en counternarratives_conan_list_sin_repetidos (si es así esta celda debería devolver true).
len(counternarratives_conan_list_sin_repetidos) == len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0])

True

In [57]:
#Add all pairs to a list with their cosine similarity score
all_sentence_combinations_cercano_a_contranarrativa_para_odio_2 = []
for i in range(0, len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0])):  # Fixme: mega importante: lo siguiente aplica para cuando estoy comparando un embedding contra todo el resto, cuando comparo todos contra todos, tengo que poner cos_sim, en vez de cos_sim[0]. Es importante pedirle la primera posición a cos_sim porque cos_sim es un tensor (una matriz, de 1xCantidadDeEmbedings contra los que estoy comparando al embedding i, por eso, si len(cos_sim[0])=1 y len(cos_sim[0])=CantidadDeEmbedings).
    all_sentence_combinations_cercano_a_contranarrativa_para_odio_2.append([cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0][i], i]) #fixme: mega importante, acá a cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan le hadrcodeo un [0], porque este es un tensor de tamaño 1x29976, si quiero acceder a la primera "fila", necesito ese [0]. Quizás haya una forma más elegante de resolverlo.

In [58]:
all_sentence_combinations_cercano_a_contranarrativa_para_odio_2;

La siguiente celda está obsoleta porque ya no ordeno todas las contranarrativas según su cos_sim, sino que directamente selecciono las top N con mayor cos_sim usando la función NmaxElements

In [59]:
#Sort list by the highest cosine similarity score
all_sentence_combinations_cercano_a_contranarrativa_para_odio_2 = sorted(all_sentence_combinations_cercano_a_contranarrativa_para_odio_2, key=lambda x: x[0], reverse=True)
counternarratives_ranking_list =[]

print("Ranking ordered by sentence similarity (the first 10 results):")
for score, i in all_sentence_combinations_cercano_a_contranarrativa_para_odio_2[0:10]: #fixme: mega importante, chequear que el rango [0:100] son 100 resultados y no 101.
    counternarratives_ranking_list.append(counternarratives_conan_list_sin_repetidos[i])
    print("{} \t {:.4f}".format(counternarratives_conan_list_sin_repetidos[i], cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan[0][i])) #fixme: mega importante, acá a cos_sim_embedding_cercano_a_contranarrativa_para_odio_2_conan le hadrcodeo un [0], porque este es un tensor de tamaño 1x6803, si quiero acceder a la primera "fila", necesito ese [0]. Quizás haya una forma más elegante de resolverlo.   #fixme: '\t' es un tab space. '{}' tina
    # inputs y los coloca donde aparece el '{}' (en este caso, el primer parámetro es sentences[i]). {:.4f} es algo de la función format, que no busqué todavía.

Ranking ordered by sentence similarity (the first 10 results):
To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith. 	 0.8228
Criminals are checking on a person's faith before the commission of their crimes? Seems a lot of bother to me. 	 0.4461
Review your law lessons! Secularism guarantees the freedom to express one's religion to everyone and therefore the use of the veil. 	 0.4314
Faith is not imposed. 	 0.4278
While this story is presented as an 'Islamic issue', the OFSTED report refers to 'countless' faith schools still that still illegally maintain gender segregation - my guess is this is a cross-faith practice and not exclusive to Islamic faith schools. 	 0.4275
Women of other faiths veil, and no one is treated like being submitted. 	 0.4218
The Christian veil is however not connected with the policy as it is the same motivation. 	 0.4162
The veil is linked to a religious practice, nothing else.

## Métricas a mano para la prueba mano n1:

Busco la forma de dado un tuit de odio, obtener todas sus contranarrativas en Conan. Lo resuelvo así:

In [60]:
df_contranarrativas_en_conan_para_odio_2=sentences_conan.loc[sentences_conan['hateSpeech'] == 'In Birmingham there is a school where girls and boys are separeted even if it isn\'t legal.', 'counterSpeech']
df_contranarrativas_en_conan_para_odio_2

1     To be fair, the OFSTED report is more concerne...
4     While this story is presented as an 'Islamic i...
7     The quoted report refers to 'countless faith s...
10    Gender segregation in faith schools is clearly...
Name: counterSpeech, dtype: object

Busco forma de chequear si una frase está en un dataframe:

In [61]:
'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.' in df_contranarrativas_en_conan_para_odio_2.values

True

### Métrica 1: cuántos de las top 10 contranarrativas sugeridas en el ranking, efectivamente aparecen en el Conan como contranarrativas para el discurso de odio_2

In [62]:
metrica1 = 0;
for i in range(0,len(counternarratives_ranking_list)):
  if counternarratives_ranking_list[i] in df_contranarrativas_en_conan_para_odio_2.values :
      metrica1 += 1;
print('En la prueba a mano el ranking eligió',metrica1, 'de las', df_contranarrativas_en_conan_para_odio_2.shape[0],'contranarrativas que existen en el Conan')

En la prueba a mano el ranking eligió 2 de las 4 contranarrativas que existen en el Conan


Hago el chequeo a mano y corroboro que la métrica está bien.

# Parametrización de funciones de prueba de hipótesis a mano para probar en todos los casos:

## Hago una prueba a mano del cuerpo del for (para chequear que me da igual a la sección "Prueba a mano n1"):

In [63]:
#Busco el índice de contranarrativa_1.
i = counternarratives_conan_list_sin_repetidos.index('To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.')

In [64]:
# La contranarrativa i es:
counternarratives_conan_list_sin_repetidos[i]

'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.'

In [65]:
#selecciono el embedding de la contranarrativa_i
contranarrativa_i_embedding_creado_por_lista = embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[i];

In [66]:
#Chequeo si los siguientes dos embeddings son iguales (espero que lo sean):
np.array_equal(contranarrativa_i_embedding_creado_por_lista, contranarrativa_1_embedding_creado_por_lista)

True

In [67]:
# Busco los discursos de odio para la contranarrativa_i
df_oido_en_conan_para_contranarrativa_i = sentences_conan.loc[sentences_conan['counterSpeech'] == counternarratives_conan_list_sin_repetidos[i], 'hateSpeech']
df_oido_en_conan_para_contranarrativa_i

0    According to a recent OFSTED report, a school ...
1    In Birmingham there is a school where girls an...
2    A school in Birningham is still separating gir...
Name: hateSpeech, dtype: object

In [68]:
# Elijo uno de los discursos de odio para la conranarrativa_i (la llamo odio_i). 
# Fixme: acá tengo más de una opción para elegir un discurso de odio para la 
# contranarrativa_i (tengo que pensar de que forma voy a elegir entre las diferentes opciones),
# acá estoy eligiendo el hatespeech 0.

odio_i = df_oido_en_conan_para_contranarrativa_i.values[0]
odio_i



'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.'

In [69]:
# Busco el embedding de uno de los mensajes de odio de los que aparecen junto a contranarrativa_i en Conan 
# (fixme: para una contranarrativa puede haber varios mensajes de odio, decidir que hago todos ellos).

# Busco el índice de odio_i.
indice_odio_i = hate_speech_conan_list_sin_repetidos.index(odio_i)

# Busco el embedding para odio_i
odio_i_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[indice_odio_i]

In [70]:
#chequeo si odio_i == odio_1
hate_speech_conan_list_sin_repetidos[indice_odio_i] == hate_speech_conan_list_sin_repetidos[728]

False

In [71]:
#Chequeo si el embeddings de odio_i y de odio_1 son iguales (espero que lo sean):
np.array_equal(odio_i_embedding_creado_por_lista, odio_1_embedding_creado_por_lista)

True

In [72]:
# Resto el embedding de odio_i al de contranarrativa_i.
embedding_contranarrativa_i_sin_discurso_de_odio_i = contranarrativa_i_embedding_creado_por_lista - odio_i_embedding_creado_por_lista

In [73]:
#Chequeo si embedding_contranarrativa_i_sin_discurso_de_odio_i y embedding_contranarrativa_1_sin_discurso_de_odio_1 son iguales (espero que lo sean):
np.array_equal(embedding_contranarrativa_i_sin_discurso_de_odio_i, embedding_contranarrativa_1_sin_discurso_de_odio_1)


True

In [74]:
# Elijo un mensaje odio_k (distinto a odio_i)
# Fixme: acá puedo elegir cualquier mensaje de odio, quisiera que no sea odio_i, tengo que 
# eleigr una forma de elegir a odio_k
odio_k = hate_speech_conan_list_sin_repetidos[k]
odio_k

"In Birmingham there is a school where girls and boys are separeted even if it isn't legal."

In [75]:
# Busco el embedding de odio_k embedding
odio_k_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[k];

In [76]:
#Chequeo si el embeddings de odio_k y de odio_2 son iguales (espero que lo sean):
np.array_equal(odio_k_embedding_creado_por_lista, odio_2_embedding_creado_por_lista)

True

In [77]:
# Sumo el embedding de odio_K a embedding_contranarrativa_i_sin_discurso_de_odio_i.
embedding_cercano_a_contranarrativa_para_odio_k = embedding_contranarrativa_i_sin_discurso_de_odio_i + odio_k_embedding_creado_por_lista

In [78]:
#Chequeo si el embedding_cercano_a_contranarrativa_para_odio_k y embedding_cercano_a_contranarrativa_para_odio_2 son iguales (espero que lo sean):
np.array_equal(embedding_cercano_a_contranarrativa_para_odio_k, embedding_cercano_a_contranarrativa_para_odio_2)

True

In [79]:
# Calculo la similaridad coseno entre el resultado del paso anterior (embedding_cercano_a_contranarrativa_para_odio_k) y los embeddings de todas las contranarrativas del Conan (embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista).
cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = util.cos_sim(embedding_cercano_a_contranarrativa_para_odio_k, embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations_cercano_a_contranarrativa_para_odio_k = []
for h in range(0, len(cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0])):  # Fixme: mega importante: lo siguiente aplica para cuando estoy comparando un embedding contra todo el resto, cuando comparo todos contra todos, tengo que poner cos_sim, en vez de cos_sim[0]. Es importante pedirle la primera posición a cos_sim porque cos_sim es un tensor (una matriz, de 1xCantidadDeEmbedings contra los que estoy comparando al embedding h, por eso, si len(cos_sim[0])=1 y len(cos_sim[0])=CantidadDeEmbedings).
    all_sentence_combinations_cercano_a_contranarrativa_para_odio_k.append([cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0][h], h]) #fixme: mega importante, acá a cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan le hadrcodeo un [0], porque este es un tensor de tamaño 1x29976, si quiero acceder a la primera "fila", necesito ese [0]. Quizás haya una forma más elegante de resolverlo

#Sort list by the highest cosine similarity score
all_sentence_combinations_cercano_a_contranarrativa_para_odio_k = sorted(all_sentence_combinations_cercano_a_contranarrativa_para_odio_k, key=lambda x: x[0], reverse=True)


counternarratives_ranking_list =[]
for score, l in all_sentence_combinations_cercano_a_contranarrativa_para_odio_k[0:10]: #fixme: mega importante, chequear que el rango [0:10] son 10 resultados y no 11.
    counternarratives_ranking_list.append(counternarratives_conan_list_sin_repetidos[l])

df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == hate_speech_conan_list_sin_repetidos[k], 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente "hate_speech_conan_list_sin_repetidos[k]"
print(df_contranarrativas_en_conan_para_odio_k)

metrica1 = 0;
for m in range(0,len(counternarratives_ranking_list)):
  if counternarratives_ranking_list[m] in df_contranarrativas_en_conan_para_odio_k.values :
      metrica1 += 1;
print('En la prueba a mano del cuerpo del for para el mensaje de odio número', k, 'el ranking eligió', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el Conan')

1     To be fair, the OFSTED report is more concerne...
4     While this story is presented as an 'Islamic i...
7     The quoted report refers to 'countless faith s...
10    Gender segregation in faith schools is clearly...
Name: counterSpeech, dtype: object
En la prueba a mano del cuerpo del for para el mensaje de odio número 1 el ranking eligió 2 de las 4 contranarrativas que existen en el Conan


# Métrica 1:

In [80]:
"""
#Chequeo cómo iterar en un dataframe:
for ind in df_oido_en_conan_para_contranarrativa_i.index:
    print(df_oido_en_conan_para_contranarrativa_i.loc[ind])
    print(ind)
"""

'\n#Chequeo cómo iterar en un dataframe:\nfor ind in df_oido_en_conan_para_contranarrativa_i.index:\n    print(df_oido_en_conan_para_contranarrativa_i.loc[ind])\n    print(ind)\n'

## Funciones útiles para calclar métrica 1:

### Nmaxelements:

In [81]:
# Function returns N largest elements and the position they have in the input list.
def Nmaxelements(list1, N):
    final_list = []
    posicionesConMaximos = set()

    for i in range(0, N):
        max1 = 0

        for j in range(len(list1)):    
            if ((list1[j] >= max1) and (j not in posicionesConMaximos)):
                max1 = list1[j];
                indMax1 = j;
                
        posicionesConMaximos.add(indMax1)
        final_list.append((max1, indMax1))
         
    return(final_list)

###### Sanity checks para Nmaxelements

In [82]:
# Chequeo si Nmaxelements devuelve lo pedido.
# Driver code
list1 = [2, 6, 41, 85, 0, 3, 7, 6, 10]
N = 9
 
# Calling the function
Nmaxelements(list1, N)

[(85, 3), (41, 2), (10, 8), (7, 6), (6, 7), (6, 1), (3, 5), (2, 0), (0, 4)]

Chequeo si el ranking formado por Nmaxelements tiene los mismos elementos que el ranking que formaba ordenando todos los elementos. Efectivamente berifico que devuelven los mismos elementos.

In [83]:
listCos = cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0].tolist()

In [84]:
print(Nmaxelements(listCos, 10))

[(0.8228143453598022, 0), (0.44610679149627686, 54), (0.43136531114578247, 1313), (0.42780253291130066, 1875), (0.4275047183036804, 1), (0.4217926263809204, 1803), (0.41619032621383667, 2173), (0.3892439603805542, 2177), (0.3875080943107605, 6038), (0.38569727540016174, 1988)]


In [85]:
all_sentence_combinations_cercano_a_contranarrativa_para_odio_k[0:10]

[[tensor(0.8228), 0],
 [tensor(0.4461), 54],
 [tensor(0.4314), 1313],
 [tensor(0.4278), 1875],
 [tensor(0.4275), 1],
 [tensor(0.4218), 1803],
 [tensor(0.4162), 2173],
 [tensor(0.3892), 2177],
 [tensor(0.3875), 6038],
 [tensor(0.3857), 1988]]

## Aplico métrica 1 usando producto matricial:

Voy a calcular todos los embeddings a mano (contranarrativa_i - odio_i + odio_k) y los voy a guardar en una matriz (en un tensor de 2 dimensiones (cantidad_de_mbeddings_calculados_a_mano x tamaño_embedding). Luego voy a llamar a cos_sim con esta matriz como primer parámetro parámetro y los embeddings de todas las contranarrativas como segúndo parámetro, espero que me devuelva una matriz de dos dimensiones (cantidad_de_mbeddings_calculados_a_mano x cantidad_contranarrativas), donde cada posición indique la cos_sim entre el embedding_calculado_a_mano_i y la contranarrativa_j.

In [86]:
print('Calculando métrica 1 usando producto matricial')

Calculando métrica 1 usando producto matricial


Fixme: en el siguiente método, lo que viene después de 

print('Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...')

Demora entre 1000 y 46396 veces más que lo que viene antes (demora 46396 veces más cuando son 100 odio y 100 contranarrat).


**100 contranarrat y 100 odio: demora 3 min (en Colab). Detalles:**

Calculando métrica 1, iteración número 0
Calculando métrica 1, iteración número 10000
Calculando métrica 1, iteración número 20000
El cálculo de cos_sim usando el método matricial: 3.741ms
Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...
La métrica 1 con el método matricial, demora: 174345.443ms. Compuestos por:
El cálculo de cos_sim usando el método matricial: 3.741ms
El armado de la métrica 1: 173568.804ms. Que a su vez, está compesta por:
La primera parte, demora: 9040.390ms.
Todos los llamados a Nmaxelements, demoran: 108469.833ms.
La segunda parte, demora: 2016.185ms.
Busqueda contranarrativas para todos los odio_k, demora: 45944.288ms.
Contar aciertos de la métrica 1, demora: 5540.396ms.

**100 contranarrat y 100 odio (en mi compu), demora 1:44min. Detalles:**

El cálculo de cos_sim usando el método matricial: 1.943ms
Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...
La métrica 1 con el método matricial, demora: 104272.478ms. Compuestos por:
El cálculo de cos_sim usando el método matricial: 1.943ms
El armado de la métrica 1: 103875.651ms. Que a su vez, está compesta por:
La primera parte, demora: 3248.946ms.
Todos los llamados a Nmaxelements, demoran: 77209.142ms.
La segunda parte, demora: 593.886ms.
Busqueda contranarrativas para todos los odio_k, demora: 19103.161ms.
Contar aciertos de la métrica 1, demora: 3020.811ms.


**100 contranarrat y todos los odio: demora 15 min (en Colab). Detalles:**

El cálculo de cos_sim usando el método matricial: 6.139ms
Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...
La métrica 1 con el método matricial, demora: 953376.638ms. Compuestos por:
El cálculo de cos_sim usando el método matricial: 6.139ms
El armado de la métrica 1: 951721.669ms. Que a su vez, está compesta por:
La primera parte, demora: 32897.451ms.
Todos los llamados a Nmaxelements, demoran: 645027.737ms.
La segunda parte, demora: 7062.055ms.
Busqueda contranarrativas para todos los odio_k, demora: 200154.156ms.
Contar aciertos de la métrica 1, demora: 27757.961ms.

**100 contranarrat y todos los odio: demora 16:29 min min (en mi compu). Detalles:**
El cálculo de cos_sim usando el método matricial: 22.359ms
Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...
La métrica 1 con el método matricial, demora: 989372.569ms. Compuestos por:
El cálculo de cos_sim usando el método matricial: 22.359ms
El armado de la métrica 1: 987132.712ms. Que a su vez, está compesta por:
La primera parte, demora: 31965.642ms.
Todos los llamados a Nmaxelements, demoran: 728649.789ms.
La segunda parte, demora: 5907.041ms.
Busqueda contranarrativas para todos los odio_k, demora: 185458.379ms.
Contar aciertos de la métrica 1, demora: 27927.447ms.

**200 contranarrat y todos los odio: demora 34 min (en Colab). Detalles:**

El cálculo de cos_sim usando el método matricial: 18.675ms
Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...
La métrica 1 con el método matricial, demora: 2057579.552ms. Compuestos por:
El cálculo de cos_sim usando el método matricial: 18.675ms
El armado de la métrica 1: 2053466.698ms. Que a su vez, está compesta por:
La primera parte, demora: 97328.012ms.
Todos los llamados a Nmaxelements, demoran: 1326219.087ms.
La segunda parte, demora: 20366.521ms.
Busqueda contranarrativas para todos los odio_k, demora: 460403.959ms.
Contar aciertos de la métrica 1, demora: 62495.830ms.

In [87]:
# Hago el setup de algunos parámetros:

#Borrar metrica1.csv si ya está creado (pensar si lo tengo que borrar).
start_calculo_metrica_1_matricial = time.time()
lista_embeddings_calculados_a_mano = []

lista_pares_métrica_1_top10_matricial = [] 

lista_pares_métrica_1_top10_random = []

indices_contranarrativa_i_odio_i_odio_k = []

iteraciones = 0;

# fixme: las siguientes listas son sólo para sanity check, una vez realizados los chequeos, se pueden comentar:
lista_contranarrativa_i_embedding_matricial = []
df_odio_en_conan_list_matricial = []
odio_i_lista_sanity_check_matricial = []
lista_listas_odio_i_embedding_matricial = []
lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial = []
lista_listas_listas_odio_k_embedding_creado_por_lista_matricial = []
#Fixme: cuando esté terminado unificar los nombres de los índices.

#fixme: si lo corro desde el script en mi compu, puedo setear hasta:
  # cantidad_contranarrativas_iteradas_matricial = 250
  # cantidad_discursos_odio_iterados_matricial = len(hate_speech_conan_list_sin_repetidos)
#fixme: en google colab puede correr hasta:
  # cantidad_contranarrativas_iteradas_matricial = 200 (quizás puede un poco más, pero hasta 250 no llega).
  # cantidad_discursos_odio_iterados_matricial = len(hate_speech_conan_list_sin_repetidos)
cantidad_contranarrativas_iteradas_matricial = 10  # fixme: el máximo es len(counternarratives_conan_list_sin_repetidos) #junto a cantidad_discursos_odio_iterados_matricial determina la parte del dataset en la que se aplica la métrica 1.
cantidad_discursos_odio_iterados_matricial = 10  # fixme: el máximo es len(hate_speech_conan_list_sin_repetidos)     # junto a cantidad_contranarrativas_iteradas_matricial determina la parte del dataset en la que se aplica la métrica 1.

#fixme: partición for: este for iteraba 6803 veces
for contranarrativa_i_indice in range(0, cantidad_contranarrativas_iteradas_matricial): #fixme: lo que modifico es esto, antes decía: for contranarrativa_i_indice in range(0, len(counternarratives_conan_list_sin_repetidos)):
  # store iteration start timestamp
  start_calculo_cos_sim = time.time()
  
  # contranarrativa_i_indice será el índice de la contranarrativa_i.
  #selecciono el embedding de la contranarrativa_i
  contranarrativa_i_embedding_creado_por_lista = embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[contranarrativa_i_indice];
  lista_contranarrativa_i_embedding_matricial.append(contranarrativa_i_embedding_creado_por_lista) # fixme: sólo para sanity check

  # Busco los discursos de odio para la contranarrativa_i
  df_oido_en_conan_para_contranarrativa_i = sentences_conan.loc[sentences_conan['counterSpeech'] == counternarratives_conan_list_sin_repetidos[contranarrativa_i_indice], 'hateSpeech']
  df_odio_en_conan_list_matricial.append(df_oido_en_conan_para_contranarrativa_i) # fixme: sólo para sanity check

  # Elijo cada uno de los discursos de odio que aparecen en Conan para la conranarrativa_i (los llamo odio_i). 
  #fixme: partición for: este for va variadno, pero más o menos itera 4 veces.
  lista_odio_i_embedding_matricial = [] #fixme: solo para sanity check
  lista_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial = []   #fixme: solo para sanity check
  lista_odio_i_para_contranarrat_i_matricial = [] # fixme: solo para sanity check
  lista_listas_odio_k_embedding_creado_por_lista_matricial = [] # fixme: solo para sanity check
  
  for ind in  df_oido_en_conan_para_contranarrativa_i.index:
    odio_i = df_oido_en_conan_para_contranarrativa_i.loc[ind]
    lista_odio_i_para_contranarrat_i_matricial.append(odio_i) # fixme: sólo para sanity check

    # Busco el índice de odio_i en hate_speech_conan_list_sin_repetidos.
    indice_odio_i = hate_speech_conan_list_sin_repetidos.index(odio_i)

    # Busco el embedding para odio_i  
    odio_i_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[indice_odio_i]
    lista_odio_i_embedding_matricial.append(odio_i_embedding_creado_por_lista) #fixme: solo para sanity check

    # Resto el embedding de odio_i al de contranarrativa_i.
    embedding_contranarrativa_i_sin_discurso_de_odio_i = contranarrativa_i_embedding_creado_por_lista - odio_i_embedding_creado_por_lista
    lista_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial.append(embedding_contranarrativa_i_sin_discurso_de_odio_i) #fixme: solo para sanity check

    lista_odio_k_embedding_creado_por_lista_matricial = [] # fixme: solo para sanity check
    #fixme: partición for: este for itera 856 veces.
    for odio_k_indice in range(0, cantidad_discursos_odio_iterados_matricial): # como máximo puede iterar hasta len(hate_speech_conan_list_sin_repetidos)
      if odio_k_indice == indice_odio_i: continue # Elijo un mensaje odio_k (distinto a odio_i)
      odio_k = hate_speech_conan_list_sin_repetidos[odio_k_indice]

      # Busco el embedding de odio_k embedding
      odio_k_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[odio_k_indice];
      lista_odio_k_embedding_creado_por_lista_matricial.append(odio_k_embedding_creado_por_lista) #fixme: solo para sanity check

      # Sumo el embedding de odio_K a embedding_contranarrativa_i_sin_discurso_de_odio_i.
      embedding_cercano_a_contranarrativa_para_odio_k = embedding_contranarrativa_i_sin_discurso_de_odio_i + odio_k_embedding_creado_por_lista

      # Appendeo el embedding calculado en el paso anterior al tensor de embeddings:
      lista_embeddings_calculados_a_mano.append(embedding_cercano_a_contranarrativa_para_odio_k)

      # Guardo una tripla con los ínidces de las contranarrativas y discursos de odio que se usaron en esta iteración:
      # (contranarrativa_i, odio_i, odio_k)
      indices_contranarrativa_i_odio_i_odio_k.append((contranarrativa_i_indice, indice_odio_i, odio_k_indice))
      
      #Imprimo estatus de estar corriendo:
      if(iteraciones % 10000 == 0):
          print('Calculando métrica 1, iteración número', iteraciones);
      iteraciones += 1;
    lista_listas_odio_k_embedding_creado_por_lista_matricial.append(lista_odio_k_embedding_creado_por_lista_matricial) # fixme: solo para sanity check
  
  lista_listas_listas_odio_k_embedding_creado_por_lista_matricial.append(lista_listas_odio_k_embedding_creado_por_lista_matricial) # fixme: solo para sanity check
  odio_i_lista_sanity_check_matricial.append(lista_odio_i_para_contranarrat_i_matricial) # fixme: sólo para sanity check
  lista_listas_odio_i_embedding_matricial.append(lista_odio_i_embedding_matricial) #fixme: sólo para sanity check
  lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial.append(lista_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial) #fixme: sólo para sanity check

# store iteration end timestamp
end_calculo_cos_sim = time.time()  

# store iteration start timestamp
start_armado_de_metrica_1 = time.time()

# transformo la lista de embeddings en una matriz de embeddings (pytorch.tensor)
matriz_de_embeddings_calculados_a_mano = torch.stack(lista_embeddings_calculados_a_mano,0)

# Elimino la lista de embeddings para liberar memoria
del(lista_embeddings_calculados_a_mano)

#Fixme: la función util.cos_sim es lo que más demora, asegurarse de que se calcula en la GPU.
# Calculo la cos_sim utilizando la matriz de embeddings
cos_sim_calculado_con_matriz = util.cos_sim(matriz_de_embeddings_calculados_a_mano, embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)

#Elimnimo la matriz de embeddings
del(matriz_de_embeddings_calculados_a_mano)

#fixme: sólo para medir tiempos, una vez optimizado, borrar
demora_busqueda_contranarrativas_odio_k = 0
demora_llamado_a_Nmaxelements = 0
demora_primera_parte_calculo_metrica_1_matricial = 0
demora_segunda_parte_calculo_metrica_1_matricial = 0
demora_contar_aciertos_metrica_1 = 0

print(f"El cálculo de cos_sim usando el método matricial: {(end_calculo_cos_sim-start_calculo_cos_sim)*10**3:.03f}ms")
print('Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...')

#Itero por la matriz_de_embeddings_calculados_a_mano armando los rankings:

# writing metrica1 to csv file:
# field names
fields = ['Contranarrativa_i', 'Odio_i', 'Odio_k', 'Contranarrativas para Odio_k en el ranking', 'Cantidad de contranarrativas que existen para Odio_k'] 

# name of csv file
filename1 = "metrica1Top10.csv"
filename2 = "metrica1Top10Random.csv"

with open(filename1, 'w') as csvfile:
  # creating a csv writer object
  csvwriterMetrica1Top10 = csv.writer(csvfile)
  # writing the fields
  csvwriterMetrica1Top10.writerow(fields)
 
  #Fixme: esto está feo, ver de refactorear (Problema: quiero escribir en múltiples csv a la vez, tengo que crear los distintos csv writers, me gustaría que )
  with open(filename2, 'w') as csvfile:
    # creating a csv writer object
    csvwriterMetrica1Top10Random = csv.writer(csvfile)
    # writing the fields
    csvwriterMetrica1Top10Random.writerow(fields)

    for indice_tripla_i in range(0,len(indices_contranarrativa_i_odio_i_odio_k)):
      start_primera_parte_calculo_metrica_1_matricial = time.time()
      # Busco el tensor correspondiente a la i_ésima tripla (contranarrativa_i, odio_i, odio_k). 
      cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_y_tripla_i = cos_sim_calculado_con_matriz[indice_tripla_i]

      #Elimino cos_sim de esta iteración: Fixe: cuando termine de iterar por toda cos_sim_calculado_con_matriz generando los rankings, la tengo que eliminar
      #del(cos_sim_calculado_con_matriz)

      #transformo el tensor con las cos_sim en una lista con las cos_sim (quizás esta transformación podría evitarse, pero me parece más claro cómo eliminar un elemento de una lsita que de un tensor).
      lista_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_y_tripla_i = cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_y_tripla_i.tolist()

      end_primera_parte_calculo_metrica_1_matricial = time.time()
      demora_primera_parte_calculo_metrica_1_matricial += (end_primera_parte_calculo_metrica_1_matricial-start_primera_parte_calculo_metrica_1_matricial)

      # store iteration start timestamp
      start_llamado_a_Nmaxelements = time.time()
      # Calculo la cos_sim e índice de las top 10 contranarrativas para odio k (calculadas a partir de la tripla (contranarrativa_i, odio_i, odio_k)).
      ranking10MejoresContranarrativasParaOdioKYTriplaI = Nmaxelements(lista_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_y_tripla_i, 10)
      # store iteration end timestamp
      end_llamado_a_Nmaxelements = time.time()
      #Calculo la demora de labúsqueda de contranarrativas para odio_k para esta iteración.
      demora_llamado_a_Nmaxelements += end_llamado_a_Nmaxelements-start_llamado_a_Nmaxelements

      start_segunda_parte_calculo_metrica_1_matricial = time.time()

      # Calculo la cos_sim e índice de las top 5 contranarrativas para odio k.
      ranking5MejoresContranarrativasParaOdioKYTriplaI = ranking10MejoresContranarrativasParaOdioKYTriplaI[0:5]

      # Calculo la cos_sim e índice de las top 3 contranarrativas para odio k.
      ranking3MejoresContranarrativasParaOdioKYTriplaI = ranking10MejoresContranarrativasParaOdioKYTriplaI[0:3]

      # Calculo la cos_sim e índice de la top 1 contranarrativa para odio k.
      ranking1MejoresContranarrativasParaOdioKYTriplaI = ranking10MejoresContranarrativasParaOdioKYTriplaI[0]

      # Busco las top 10 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top10_matricial =[]
      for score, l in ranking10MejoresContranarrativasParaOdioKYTriplaI: #fixme: mega importante, chequear que el rango [0:10] son 10 resultados y no 11.
          counternarratives_ranking_list_top10_matricial.append(counternarratives_conan_list_sin_repetidos[l])

      # Busco las top 5 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top5_matricial = counternarratives_ranking_list_top10_matricial[0:5]

      # Busco las top 3 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top3_matricial = counternarratives_ranking_list_top10_matricial[0:3]

      # Busco la top 1 contranarrativa (busco efectivamente el string, no el índices).
      counternarratives_ranking_list_top1_matricial = counternarratives_ranking_list_top10_matricial[0:1]

      #Fixme: acá puedo eliminar código repetido:

      #Genero un top 10 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top10_random_matricial = random.sample(counternarratives_conan_list_sin_repetidos, 10)

      #Genero un top 5 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top5_random = counternarratives_ranking_list_top10_random_matricial[0:5]

      #Genero un top 3 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top3_random =  counternarratives_ranking_list_top10_random_matricial[0:3]

      #Genero un top 1 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top1_random =  counternarratives_ranking_list_top10_random_matricial[0:1]

      #Fixme: acá puedo eliminar código repetido:
      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 10 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top10. 
      # Además calclo la métrica 1 para el ranking de 10 contranarrativas random.

      # Primero calculo los indices de contranarrativa_i, odio_i y odio_k a partir de las triplas
      tripla_iterada = indices_contranarrativa_i_odio_i_odio_k[indice_tripla_i]
      indice_contranarrativa_i = tripla_iterada[0]
      indice_odio_i = tripla_iterada[1]
      indice_odio_k = tripla_iterada[2]
      odio_k = hate_speech_conan_list_sin_repetidos[indice_odio_k]

      end_segunda_parte_calculo_metrica_1_matricial = time.time()
      demora_segunda_parte_calculo_metrica_1_matricial += (end_segunda_parte_calculo_metrica_1_matricial - start_segunda_parte_calculo_metrica_1_matricial)


      # store iteration start timestamp
      start_busqueda_contranarrativas_para_odio_k = time.time()
      df_contranarrativas_en_conan_para_odio_k_matricial = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k_sacado_de_lista_de_triplas.
      # store iteration end timestamp
      end_busqueda_contranarrativas_para_odio_k = time.time()
      #Calculo la demora de labúsqueda de contranarrativas para odio_k para esta iteración.
      demora_busqueda_contranarrativas_odio_k += end_busqueda_contranarrativas_para_odio_k-start_busqueda_contranarrativas_para_odio_k
      
      start_contar_aciertos_metrica_1 = time.time()
      metrica1_matricial = 0;
      metrica1_random_top10_matricial = 0;
      for m in range(0,len(counternarratives_ranking_list_top10_matricial)):
        if counternarratives_ranking_list_top10_matricial[m] in df_contranarrativas_en_conan_para_odio_k_matricial.values:
            metrica1_matricial += 1;
        if counternarratives_ranking_list_top10_random_matricial[m] in df_contranarrativas_en_conan_para_odio_k_matricial.values:
            metrica1_random_top10_matricial += 1;
      end_contar_aciertos_metrica_1 = time.time()
      demora_contar_aciertos_metrica_1 += (end_contar_aciertos_metrica_1 - start_contar_aciertos_metrica_1)

      lista_pares_métrica_1_top10_matricial.append(('Contranarrativa_i, está en la posición', indice_contranarrativa_i,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', indice_odio_k,' en hate_speech_conan_list_sin_repetidos, el ranking de 10 contranarrativas contiene', metrica1_matricial, 'de las', df_contranarrativas_en_conan_para_odio_k_matricial.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      lista_pares_métrica_1_top10_random.append(('Contranarrativa_i, está en la posición', indice_contranarrativa_i,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', indice_odio_k,' en hate_speech_conan_list_sin_repetidos, el ranking de 10 contranarrativas random contiene', metrica1_random_top10_matricial, 'de las', df_contranarrativas_en_conan_para_odio_k_matricial.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))
      #################
      # Write thi iteration into a .csv:
      # creating this iteration row:
      row_metrica_1_top_10_matricial = [indice_contranarrativa_i, indice_odio_i, indice_odio_k, metrica1_matricial, df_contranarrativas_en_conan_para_odio_k_matricial.shape[0]]
      row_metrica_1_top_10_random = [indice_contranarrativa_i, indice_odio_i, indice_odio_k, metrica1_random_top10_matricial, df_contranarrativas_en_conan_para_odio_k_matricial.shape[0]]
      
      # writing this iteration rows:
      csvwriterMetrica1Top10.writerow(row_metrica_1_top_10_matricial)
      csvwriterMetrica1Top10Random.writerow(row_metrica_1_top_10_random)
      #################
    # store iteration end timestamp
    end_armado_de_metrica_1 = time.time()  

    end_calculo_metrica_1_matricial = time.time()
    print(f"La métrica 1 con el método matricial, demora: {(end_calculo_metrica_1_matricial-start_calculo_metrica_1_matricial)*10**3:.03f}ms. Compuestos por:")

    print(f"El cálculo de cos_sim usando el método matricial: {(end_calculo_cos_sim-start_calculo_cos_sim)*10**3:.03f}ms")
    print(f"El armado de la métrica 1: {(end_armado_de_metrica_1-start_armado_de_metrica_1)*10**3:.03f}ms. Que a su vez, está compesta por:")
    print(f"La primera parte, demora: {(demora_primera_parte_calculo_metrica_1_matricial)*10**3:.03f}ms.")
    print(f"Todos los llamados a Nmaxelements, demoran: {(demora_llamado_a_Nmaxelements)*10**3:.03f}ms.")
    print(f"La segunda parte, demora: {(demora_segunda_parte_calculo_metrica_1_matricial)*10**3:.03f}ms.")
    print(f"Busqueda contranarrativas para todos los odio_k, demora: {(demora_busqueda_contranarrativas_odio_k)*10**3:.03f}ms.")
    print(f"Contar aciertos de la métrica 1, demora: {(demora_contar_aciertos_metrica_1)*10**3:.03f}ms.")


      #Fixme: hacer funcionar el ranking Top 10 maricial y dps escribir funciones para generar el resto.

Calculando métrica 1, iteración número 0
El cálculo de cos_sim usando el método matricial: 2.230ms
Matriz de cos_sim calculada con el método matricial, armando el resto de la métrica 1...
La métrica 1 con el método matricial, demora: 1991.278ms. Compuestos por:
El cálculo de cos_sim usando el método matricial: 2.230ms
El armado de la métrica 1: 1961.073ms. Que a su vez, está compesta por:
La primera parte, demora: 81.744ms.
Todos los llamados a Nmaxelements, demoran: 1227.308ms.
La segunda parte, demora: 19.531ms.
Busqueda contranarrativas para todos los odio_k, demora: 524.770ms.
Contar aciertos de la métrica 1, demora: 66.465ms.


### Leo métrica 1 desde los archivos .csv

In [88]:
# En esta celda levanto el csv y lo guardo en una lista de listas.
# csv file name
filename = "metrica1Top10.csv"
 
# initializing the titles and rows list
fields = []
rows = []

# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)
 
    # get total number of rows
    print("Total no. of rows: %d"%(csvreader.line_num))
    
    # Armo una lista de lsitas con todos los elementos del csv:
    lista_pares_métrica_1_top10_matricial_leida_de_csv = [];
    for row in rows:
        # parsing each column of a row
        rowList = [];
        for col in row:
            rowList.append(int(col));
        lista_pares_métrica_1_top10_matricial_leida_de_csv.append(rowList);

Total no. of rows: 271


## Aplico métrica 1 con un for-loop:

In [89]:
print('Calculando métrica 1')

Calculando métrica 1


La versión con for-loop, para 100 contranarrat y 100 odio, demora 4.8 min.

In [90]:
# Hago el setup de algunos parámetros:
lista_pares_métrica_1_top10 = [] 
lista_pares_métrica_1_top5 = []
lista_pares_métrica_1_top3 = []
lista_pares_métrica_1_top1 = []

lista_pares_métrica_1_top10_random = []
lista_pares_métrica_1_top5_random = []
lista_pares_métrica_1_top3_random = []
lista_pares_métrica_1_top1_random = []

# fixme: inicio: las siguientes listas son solo para sanity check, una vez realizados los chequeos, se pueden comentar:
df_odio_en_conan_list_for_loop = []
lista_contranarrativa_i_embedding_for_loop = []
indices_contranarrativa_i_odio_i_odio_k_for_loop_soloParaSanityChequearImplementacionMatricial = []
odio_i_lista_sanity_check_for_loop = []
lista_listas_odio_i_embedding_for_loop = []
lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop =[]
lista_listas_listas_odio_k_embedding_creado_por_lista_for_loop = []
lista_listas_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = []
#### fixme: fin de: las siguientes listas son solo para sanity check, una vez realizados los chequeos, se pueden comentar: 

iteraciones = 0;

cantidad_contranarrativas_iteradas_for_loop = 10  # fixme: el máximo es len(counternarratives_conan_list_sin_repetidos) #junto a cantidad_discursos_odio_iterados_for_loop determina la parte del dataset en la que se aplica la métrica 1.
cantidad_discursos_odio_iterados_for_loop = 10    # fixme: el máximo es len(hate_speech_conan_list_sin_repetidos)     # junto a cantidad_contranarrativas_iteradas_for_loop determina la parte del dataset en la que se aplica la métrica 1.

#fixme: partición for: este for iteraba 6803 veces
for contranarrativa_i_indice in range(0, cantidad_contranarrativas_iteradas_for_loop): #fixme: lo que modifico es esto, antes decía: for contranarrativa_i_indice in range(0, len(counternarratives_conan_list_sin_repetidos)):
  # contranarrativa_i_indice será el índice de la contranarrativa_i.
  #selecciono el embedding de la contranarrativa_i
  contranarrativa_i_embedding_creado_por_lista = embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista[contranarrativa_i_indice];
  lista_contranarrativa_i_embedding_for_loop.append(contranarrativa_i_embedding_creado_por_lista)

  # Busco los discursos de odio para la contranarrativa_i
  df_oido_en_conan_para_contranarrativa_i = sentences_conan.loc[sentences_conan['counterSpeech'] == counternarratives_conan_list_sin_repetidos[contranarrativa_i_indice], 'hateSpeech']
  df_odio_en_conan_list_for_loop.append(df_oido_en_conan_para_contranarrativa_i) #fixme: solo para sanity check

  # Elijo cada uno de los discursos de odio que aparecen en Conan para la conranarrativa_i (los llamo odio_i). 
  #fixme: partición for: este for va variadno, pero más o menos itera 4 veces.
  lista_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop = []   #fixme: solo para sanity check
  lista_odio_i_para_contranarrat_i_for_loop = [] #fixme: solo para sanity check.
  lista_odio_i_embedding_for_loop = [] #fixme: solo para sanity check
  lista_listas_odio_k_embedding_creado_por_lista_for_loop = [] # fixme: solo para sanity check
  lista_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = [] #fixme: solo para sanity check

  for ind in  df_oido_en_conan_para_contranarrativa_i.index:
    odio_i = df_oido_en_conan_para_contranarrativa_i.loc[ind]
   
    lista_odio_i_para_contranarrat_i_for_loop.append(odio_i) # fixme: sólo para sanity check
    
    # Busco el índice de odio_1 en hate_speech_conan_list_sin_repetidos.
    indice_odio_i = hate_speech_conan_list_sin_repetidos.index(odio_i)

    # Busco el embedding para odio_i  
    odio_i_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[indice_odio_i]
    # Busco el embedding para odio_i  
    odio_i_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[indice_odio_i]

    lista_odio_i_embedding_for_loop.append(odio_i_embedding_creado_por_lista) #fixme: solo para sanity check

    
    # Resto el embedding de odio_i al de contranarrativa_i.
    embedding_contranarrativa_i_sin_discurso_de_odio_i = contranarrativa_i_embedding_creado_por_lista - odio_i_embedding_creado_por_lista
    lista_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop.append(embedding_contranarrativa_i_sin_discurso_de_odio_i) #fixme: solo para sanity check

    lista_odio_k_embedding_creado_por_lista_for_loop = [] # fixme: solo para sanity check
    #fixme: partición for: este for itera 856 veces.
    lista_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = []
    for odio_k_indice in range(0, cantidad_discursos_odio_iterados_for_loop): # como máximo puede iterar hasta len(hate_speech_conan_list_sin_repetidos)
      if odio_k_indice == indice_odio_i: continue # Elijo un mensaje odio_k (distinto a odio_i)
      odio_k = hate_speech_conan_list_sin_repetidos[odio_k_indice]

      # Busco el embedding de odio_k embedding
      odio_k_embedding_creado_por_lista = embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista[odio_k_indice];
      lista_odio_k_embedding_creado_por_lista_for_loop.append(odio_k_embedding_creado_por_lista) #fixme: solo para sanity check

      # Sumo el embedding de odio_K a embedding_contranarrativa_i_sin_discurso_de_odio_i.
      embedding_cercano_a_contranarrativa_para_odio_k = embedding_contranarrativa_i_sin_discurso_de_odio_i + odio_k_embedding_creado_por_lista

      #fixme: chequear el cálculo decos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan, ver que devuelve util.cos_sim. 
      # Calculo la similaridad coseno entre el resultado del paso anterior (embedding_cercano_a_contranarrativa_para_odio_k) y los embeddings de todas las contranarrativas del Conan (embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista).
      #fixme: según entiendo, por la documentación de cos_sim, este tensor tiene en la segunda posición la cos_sim entre embedding_cercano_a_contranarrativa_para_odio_k y todos los embeddings de embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista. 
      cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = util.cos_sim(embedding_cercano_a_contranarrativa_para_odio_k, embeddings_counternarratives_conan_list_sin_repetidos_creado_por_lista)
      #Guardo el resultado anterior en una lista para hacer sanity check de la implementación matricial.
      lista_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan.append(cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan)#fixme: solo para sanity check.
      

      #transformo el tensor con las cos_sim en una lista con las cos_sim (quizás esta transformación podría evitarse, pero me parece más claro cómo eliminar un elemento de una lsita que de un tensor).
      lista_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan = cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0].tolist()
      
      # Calculo la cos_sim e índice de las top 10 contranarrativas para odio k.
      ranking10MejoresContranarrativasParaOdioK = Nmaxelements(lista_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan, 10)

      # Calculo la cos_sim e índice de las top 5 contranarrativas para odio k.
      ranking5MejoresContranarrativasParaOdioK = ranking10MejoresContranarrativasParaOdioK[0:5] 

      # Calculo la cos_sim e índice de las top 3 contranarrativas para odio k.
      ranking3MejoresContranarrativasParaOdioK = ranking10MejoresContranarrativasParaOdioK[0:3]

      # Calculo la cos_sim e índice de la top 1 contranarrativa para odio k.
      ranking1MejoresContranarrativasParaOdioK = ranking10MejoresContranarrativasParaOdioK[0]

      # Busco las top 10 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top10 =[]
      for score, l in ranking10MejoresContranarrativasParaOdioK: #fixme: mega importante, chequear que el rango [0:10] son 10 resultados y no 11.
          counternarratives_ranking_list_top10.append(counternarratives_conan_list_sin_repetidos[l])

      # Busco las top 5 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top5 = counternarratives_ranking_list_top10[0:5]

      # Busco las top 3 contranarrativas (busco efectivamente los strings, no los índices).
      counternarratives_ranking_list_top3 = counternarratives_ranking_list_top10[0:3]

      # Busco la top 1 contranarrativa (busco efectivamente el string, no el índices).
      counternarratives_ranking_list_top1 = counternarratives_ranking_list_top10[0:1]

      #Fixme: acá puedo eliminar código repetido:

      #Genero un top 10 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top10_random = random.sample(counternarratives_conan_list_sin_repetidos, 10)
      
      #Genero un top 5 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top5_random = counternarratives_ranking_list_top10_random[0:5]

      #Genero un top 3 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top3_random =  counternarratives_ranking_list_top10_random[0:3]

      #Genero un top 1 con contranarrativas tomadas al azar
      counternarratives_ranking_list_top1_random =  counternarratives_ranking_list_top10_random[0:1]

      #Fixme: acá puedo eliminar código repetido:
      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 10 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top10. 
      # Además calclo la métrica 1 para el ranking de 10 contranarrativas random.

      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      metrica1_random_top10 = 0;
      for m in range(0,len(counternarratives_ranking_list_top10)):
        if counternarratives_ranking_list_top10[m] in df_contranarrativas_en_conan_para_odio_k.values:
            metrica1 += 1;
        if counternarratives_ranking_list_top10_random[m] in df_contranarrativas_en_conan_para_odio_k.values:
            metrica1_random_top10 += 1;
      lista_pares_métrica_1_top10.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 10 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))
      lista_pares_métrica_1_top10_random.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 10 contranarrativas random contiene', metrica1_random_top10, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))
      
      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 5 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top5
      # Además calclo la métrica 1 para el ranking de 5 contranarrativas random.
      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      metrica1_random_top5 = 0;
      for m in range(0,len(counternarratives_ranking_list_top5)):
        if counternarratives_ranking_list_top5[m] in df_contranarrativas_en_conan_para_odio_k.values :
            metrica1 += 1;
        if counternarratives_ranking_list_top5_random[m] in df_contranarrativas_en_conan_para_odio_k.values:
            metrica1_random_top5 += 1;
      lista_pares_métrica_1_top5.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 5 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))
      lista_pares_métrica_1_top5_random.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 5 contranarrativas random contiene', metrica1_random_top5, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 3 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top3
      # Además calclo la métrica 1 para el ranking de 3 contranarrativas random.
      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      metrica1_random_top3 = 0;
      for m in range(0,len(counternarratives_ranking_list_top3)):
        if counternarratives_ranking_list_top3[m] in df_contranarrativas_en_conan_para_odio_k.values :
            metrica1 += 1;
        if counternarratives_ranking_list_top3_random[m] in df_contranarrativas_en_conan_para_odio_k.values:
            metrica1_random_top3 += 1;
      lista_pares_métrica_1_top3.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 3 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))
      lista_pares_métrica_1_top3_random.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 3 contranarrativas random contiene', metrica1_random_top3, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      # Calculo la métrica_1 para el caso contranarrativa_i, odio_i, odio_k y ranking de 1 contranarrativas 
      # y la guardo en la lista_pares_métrica_1_top1
      # Además calclo la métrica 1 para el ranking de 1 contranarrativa random.
      df_contranarrativas_en_conan_para_odio_k = sentences_conan.loc[sentences_conan['hateSpeech'] == odio_k, 'counterSpeech'] #fixme: ojo con esto, tengo que chequear que el hate speech que me interesa es efectivamente odio_k.
      metrica1 = 0;
      metrica1_random_top1 = 0;
      for m in range(0,len(counternarratives_ranking_list_top1)):
        if counternarratives_ranking_list_top1[m] in df_contranarrativas_en_conan_para_odio_k.values :
            metrica1 += 1;
        if counternarratives_ranking_list_top1_random[m] in df_contranarrativas_en_conan_para_odio_k.values:
            metrica1_random_top1 += 1;

      lista_pares_métrica_1_top1.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 3 contranarrativas contiene', metrica1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))
      lista_pares_métrica_1_top1_random.append(('Contranarrativa_i, está en la posición', contranarrativa_i_indice,'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero', indice_odio_i, 'en hate_speech_conan_list_sin_repetidos.', 'Para el mensaje de odio en la posición', odio_k_indice,' en hate_speech_conan_list_sin_repetidos, el ranking de 1 contranarrativas random contiene', metrica1_random_top1, 'de las', df_contranarrativas_en_conan_para_odio_k.shape[0],'contranarrativas que existen en el conan para ese discurso de odio'))

      # Solo para sanity check contra método matricial:
      # Guardo una tripla con los ínidces de las contranarrativas y discursos de odio que se usaron en esta iteración:
      # (contranarrativa_i, odio_i, odio_k)
      indices_contranarrativa_i_odio_i_odio_k_for_loop_soloParaSanityChequearImplementacionMatricial.append((contranarrativa_i_indice, indice_odio_i, odio_k_indice))

      #Imprimo estatus de estar corriendo:
      if(iteraciones % 100 == 0):
          print('Calculando métrica 1, iteración número', iteraciones);
      iteraciones += 1;
    lista_listas_odio_k_embedding_creado_por_lista_for_loop.append(lista_odio_k_embedding_creado_por_lista_for_loop) # fixme: solo para sanity check
    lista_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan.append(lista_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan) # fixme: solo para sanity check

  lista_listas_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan.append(lista_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan)
  lista_listas_listas_odio_k_embedding_creado_por_lista_for_loop.append(lista_listas_odio_k_embedding_creado_por_lista_for_loop) # fixme: solo para sanity check
  odio_i_lista_sanity_check_for_loop.append(lista_odio_i_para_contranarrat_i_for_loop) # fixme: solo para sanity check
  lista_listas_odio_i_embedding_for_loop.append(lista_odio_i_embedding_for_loop) #fixme: solo para sanity check
  lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop.append(lista_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop) #fixme: solo para sanity check

Calculando métrica 1, iteración número 0
Calculando métrica 1, iteración número 100
Calculando métrica 1, iteración número 200


In [91]:
#fixme: acá printeo la lista de resultados y veo que hay algunos discursos de odio que tiene muchas contranarrativas 
# (por ejemplo 85), quizás el ranking tenga que ser de más elementos (aunque por otra parte yo quiero que el ranking
# devuelva pocos elementos y muy buenos -tengo el problema de que si el ranking devuelve 10 elementos y los 10 son
# buenos, pero para ese discurso de odio hay 85 contranarrativas, me va a quedar que el sistema devuelve 10 de las 85
# posibles contranarrativas [es un muy mal resultado]-)
# lista_pares_métrica_1_top10


##Hago sanity checks para versión matricial comparando contra la métrica 1 calculada con for-loop:

Fixme: lo siguiente es lo que tengo que terminar de calcular para que terminar de calcular la metrica 1 de forma matricial:

La siguiente celda, se debe calcular una vez obtenida la matriz con todos los embeddings calculados a mano.

In [92]:
# Chequeo que los embeddings de las contranarrativas_i son los mismos para los métodos matricial y for-loop: 

res = True
for i in range(0, min(len(lista_contranarrativa_i_embedding_matricial), len(lista_contranarrativa_i_embedding_for_loop))):
  res = res and torch.equal(lista_contranarrativa_i_embedding_matricial[i],lista_contranarrativa_i_embedding_for_loop[i])
res

True

In [93]:
# Chequeo si los discursos de odio obtenidos para la contranarrativa_i en la versión 
#matricial, son los mismos que los obtenidos para versión for-loop 

for i in range(0, min(len(df_odio_en_conan_list_for_loop), len(df_odio_en_conan_list_matricial))):
  print(df_odio_en_conan_list_matricial[i] == df_odio_en_conan_list_for_loop [i])

0    True
1    True
2    True
Name: hateSpeech, dtype: bool
3    True
4    True
5    True
Name: hateSpeech, dtype: bool
6    True
7    True
8    True
Name: hateSpeech, dtype: bool
9     True
10    True
11    True
Name: hateSpeech, dtype: bool
12    True
13    True
14    True
Name: hateSpeech, dtype: bool
15    True
16    True
17    True
Name: hateSpeech, dtype: bool
18    True
19    True
20    True
Name: hateSpeech, dtype: bool
21    True
22    True
23    True
Name: hateSpeech, dtype: bool
24    True
25    True
26    True
Name: hateSpeech, dtype: bool
27    True
28    True
29    True
Name: hateSpeech, dtype: bool


In [94]:
# Chequeo que los odio_i sean los mismos en la versión matricial y en la de for_loop:
len_matricial = len(odio_i_lista_sanity_check_matricial)
len_for_loop = len(odio_i_lista_sanity_check_for_loop)
odio_i_lista_sanity_check_matricial[0:len_for_loop] == odio_i_lista_sanity_check_for_loop[0:len_matricial]

True

In [95]:
# Chequeo que los embeddings de odio_i sean los mismos en la versión matricial y en la de for_loop:

res = True
for i in range(0, min(len(lista_listas_odio_i_embedding_matricial), len(lista_listas_odio_i_embedding_for_loop))):
  for j in range(0, min(len(lista_listas_odio_i_embedding_matricial[i]), len(lista_listas_odio_i_embedding_for_loop[i]))):
    res = res and torch.equal(lista_listas_odio_i_embedding_matricial[i][j],lista_listas_odio_i_embedding_for_loop[i][j])
res

True

In [96]:
# Chequeo si los embeddings de odio_k son los mismos en la versión matricial y en la de for loop:
lista_listas_listas_odio_k_embedding_creado_por_lista_matricial[0][0][0]

# Chequeo que los embeddings de odio_i sean los mismos en la versión matricial y en la de for_loop:

res = True
for i in range(0, min(len(lista_listas_listas_odio_k_embedding_creado_por_lista_matricial), len(lista_listas_listas_odio_k_embedding_creado_por_lista_for_loop))):
  for j in range(0, min(len(lista_listas_listas_odio_k_embedding_creado_por_lista_matricial[i]), len(lista_listas_listas_odio_k_embedding_creado_por_lista_for_loop[i]))):
    for k in range(0, min(len(lista_listas_listas_odio_k_embedding_creado_por_lista_matricial[i][j]), len(lista_listas_listas_odio_k_embedding_creado_por_lista_for_loop[i][j]))):
      res = res and torch.equal(lista_listas_listas_odio_k_embedding_creado_por_lista_matricial[i][j][k],lista_listas_listas_odio_k_embedding_creado_por_lista_for_loop[i][j][k])
res

True

In [97]:
# Chequeo que los embeddings de contranarrativa_i_sin_discurso_de_odio_i sean los mismos en la versión matricial y en la de for_loop:

res = True
for i in range(0, min(len(lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial), len(lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop))):
  res = res and print(len(lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial[i]) == len(lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop[i]))
  for j in range(0, min(len(lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial[i]), len(lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop[i]))):
    res = res and torch.equal(lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_matricial[i][j],lista_listas_embedding_contranarrativa_i_sin_discurso_de_odio_i_for_loop[i][j])
res

True


In [98]:
# Chequeo que las triplas (contranarrativa_i, odio_i, odio_k) sean los mismos en ambos métodos (matricial y for-loop) 
# Puede haber una tripla de listas más larga que otra, esto chequea que la tripla mas corta sea parte de la cabeza de la más larga.

res = True
for i in range (0, min(len(indices_contranarrativa_i_odio_i_odio_k_for_loop_soloParaSanityChequearImplementacionMatricial), len(indices_contranarrativa_i_odio_i_odio_k))):
   res = res and indices_contranarrativa_i_odio_i_odio_k_for_loop_soloParaSanityChequearImplementacionMatricial[i] == indices_contranarrativa_i_odio_i_odio_k[i]
res    

True

### Sanity check: comparo cos_sim calculada matricialmente vs calculada con for_loop:

In [99]:
# Chequeo si las cos_sim sean iguales.
print('Los dos métodos (matricial y for_loop) devuelven la misma cos_sim: ', torch.equal(lista_listas_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[0][0][0][0], cos_sim_calculado_con_matriz[0]))

# Confirmo que no lo son. Printeo ambos embeddings y los comparo a mano (los comparo en https://text-compare.com/) y verficico que hay minimas
# diferencias,sospecho que se debe a errores de redondeo de util.cos_sim.
#Las siguientes dos celdas printean los embeddings de las implementaciones for_loop y matricial:

Los dos métodos (matricial y for_loop) devuelven la misma cos_sim:  False


In [100]:
# Printeo las triplas (contranarrativa_i, odio_i, odio_k), para tener presente para cuáles triplas se 
# está calculando cos_sim en las siguientes dos celdas:
indices_contranarrativa_i_odio_i_odio_k

[(0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 0, 4),
 (0, 0, 5),
 (0, 0, 6),
 (0, 0, 7),
 (0, 0, 8),
 (0, 0, 9),
 (0, 1, 0),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 1, 6),
 (0, 1, 7),
 (0, 1, 8),
 (0, 1, 9),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 2, 6),
 (0, 2, 7),
 (0, 2, 8),
 (0, 2, 9),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 0, 4),
 (1, 0, 5),
 (1, 0, 6),
 (1, 0, 7),
 (1, 0, 8),
 (1, 0, 9),
 (1, 1, 0),
 (1, 1, 2),
 (1, 1, 3),
 (1, 1, 4),
 (1, 1, 5),
 (1, 1, 6),
 (1, 1, 7),
 (1, 1, 8),
 (1, 1, 9),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 2, 6),
 (1, 2, 7),
 (1, 2, 8),
 (1, 2, 9),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 0, 4),
 (2, 0, 5),
 (2, 0, 6),
 (2, 0, 7),
 (2, 0, 8),
 (2, 0, 9),
 (2, 1, 0),
 (2, 1, 2),
 (2, 1, 3),
 (2, 1, 4),
 (2, 1, 5),
 (2, 1, 6),
 (2, 1, 7),
 (2, 1, 8),
 (2, 1, 9),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 3),
 (2, 2, 4),
 (2, 2, 5),
 (2, 2, 6),
 (2, 2, 7),
 (2, 2, 8),
 (2, 2, 9),
 (3, 0, 1),
 (3, 0, 2),
 (3,

In [101]:
# Printeo la cos_sim calculada matricialmente de las triplas (contranarrativa_i, odio_i, odio_k) contra todas las contranarrativas del Dataset para
# mostrar que a pesar de que hay pequeños errores de redondeo, la cos_sim calculada con el método for_loop y el método matricial, dan resultados 
# prácticamente idénticos

for indice_tripla in range(0, len(indices_contranarrativa_i_odio_i_odio_k)):
  print(cos_sim_calculado_con_matriz[indice_tripla])
  # Para printear los tensores enteros, descomentar las siguientes tres líneas:
  #torch.set_printoptions(profile="full")
  #print(cos_sim_calculado_con_matriz[0])
  #torch.set_printoptions(profile="default")

tensor([0.8228, 0.4275, 0.3856,  ..., 0.2685, 0.0595, 0.1089])
tensor([0.8168, 0.4159, 0.3676,  ..., 0.1833, 0.0414, 0.1169])
tensor([0.5145, 0.1288, 0.2164,  ..., 0.2025, 0.0152, 0.1043])
tensor([0.5490, 0.1362, 0.2179,  ..., 0.2562, 0.0278, 0.0820])
tensor([ 0.5139,  0.0970,  0.1840,  ...,  0.1753, -0.0041,  0.1076])
tensor([0.5203, 0.1705, 0.2497,  ..., 0.2192, 0.0210, 0.1746])
tensor([0.5493, 0.1653, 0.2322,  ..., 0.1734, 0.0215, 0.1510])
tensor([0.5475, 0.1941, 0.2580,  ..., 0.1917, 0.0100, 0.1630])
tensor([0.5534, 0.1310, 0.2283,  ..., 0.1919, 0.0104, 0.1050])
tensor([0.8864, 0.5403, 0.4189,  ..., 0.1686, 0.0265, 0.1101])
tensor([0.8526, 0.4767, 0.3876,  ..., 0.1401, 0.0261, 0.1170])
tensor([0.5672, 0.1993, 0.2443,  ..., 0.1671, 0.0050, 0.1056])
tensor([0.5999, 0.2065, 0.2460,  ..., 0.2170, 0.0166, 0.0851])
tensor([ 0.5642,  0.1677,  0.2131,  ...,  0.1423, -0.0127,  0.1085])
tensor([0.5759, 0.2373, 0.2767,  ..., 0.1860, 0.0110, 0.1727])
tensor([0.5938, 0.2278, 0.2562,  ..., 0.141

In [102]:
# Printeo la cos_sim calculada con el for_loop, de las triplas (contranarrativa_i, odio_i, odio_k) contra todas las contranarrativas del Dataset para
# mostrar que a pesar de que hay pequeños errores de redondeo, la cos_sim calculada con el método for_loop y el método matricial, dan resultados 
# prácticamente idénticos

indice_contranarrativa_i = 0  # Toma valores en el intervalo [0:cantidad_contranarrativas_iteradas_for_loop)
indice_odio_i = 0             # Los valores que puede tomar varían según la cantidad de discursos de odio que existan para la contranarrativa_i
indice_odio_k = 0             # Toma valores en el intervalo [0:cantidad_discursos_odio_iterados_for_loop)

print('cos_sim calculada con for_loop:', lista_listas_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[indice_contranarrativa_i][indice_odio_i][indice_odio_k][0])

# Para printear el tensor entero, descomentar las siguientes líneas:
#torch.set_printoptions(profile="full")
#print(lista_listas_listas_sanity_check_cos_sim_embedding_cercano_a_contranarrativa_para_odio_k_conan[indice_contranarrativa_i][indice_odio_i][indice_odio_k][0])
#torch.set_printoptions(profile="default")


cos_sim calculada con for_loop: tensor([0.8228, 0.4275, 0.3856,  ..., 0.2685, 0.0595, 0.1089])


In [103]:
lista_pares_métrica_1_top10[0]

('Contranarrativa_i, está en la posición',
 0,
 'en counternarratives_conan_list_sin_repetidos. Odio_i está en la posición numero',
 0,
 'en hate_speech_conan_list_sin_repetidos.',
 'Para el mensaje de odio en la posición',
 1,
 ' en hate_speech_conan_list_sin_repetidos, el ranking de 10 contranarrativas contiene',
 2,
 'de las',
 4,
 'contranarrativas que existen en el conan para ese discurso de odio')

In [104]:
# Chequeo que la metrica 1 es igual calculada con cualquiera de los dos métodos.

posicion_contranarrativa_i = 1
posicion_odio_i = 3
posicion_odio_k = 6
posicion_cantidad_contranarrativas_encontradas = 8
posicion_contranarrativas_totales = 10

res = True
for i in range(0,len(lista_pares_métrica_1_top10)):
  res = res and (lista_pares_métrica_1_top10_matricial_leida_de_csv[i][0] == lista_pares_métrica_1_top10[i][posicion_contranarrativa_i] and
  lista_pares_métrica_1_top10_matricial_leida_de_csv[i][1] == lista_pares_métrica_1_top10[i][posicion_odio_i] and
  lista_pares_métrica_1_top10_matricial_leida_de_csv[i][2] == lista_pares_métrica_1_top10[i][posicion_odio_k] and 
  lista_pares_métrica_1_top10_matricial_leida_de_csv[i][3] == lista_pares_métrica_1_top10[i][posicion_cantidad_contranarrativas_encontradas] and
  lista_pares_métrica_1_top10_matricial_leida_de_csv[i][4] == lista_pares_métrica_1_top10[i][posicion_contranarrativas_totales])

res

True

In [105]:
indices_contranarrativa_i_odio_i_odio_k_for_loop_soloParaSanityChequearImplementacionMatricial == indices_contranarrativa_i_odio_i_odio_k

True

# Métrica 2:

metrica_2 = metrica_1 / min(número de contranarrativas para el un discurso de odio dado, cantidad_elementos_del_ranking).
El mínimo está porque para algunos discursos de odio, el "número de contranarrativas para el un discurso de odio dado" es mucho mayor a "cantidad_elementos_del_ranking" y esto hace parecer que el ranking funciona muy mal, cuando en realidad el tope "cantidad_elementos_del_ranking" hace que no pueda devolver mayor "número de contranarrativas para el un discurso de odio dado".

##Aplico métrica 2:

### Aplico métrica 2 leyendo desde los .csv:

In [106]:
# Métrica 2 en ranking de 10 elementos
lista_metrica_2_top10_leida_de_csv = []
posicion_contranarrativas_encontradas = 3 
posicion_contranarrativas_totales = 4
cantidad_elementos_del_rankingTop10 = 10

for i in range(0, len(lista_pares_métrica_1_top10_matricial_leida_de_csv)):
  metrica_2_para_par_i = lista_pares_métrica_1_top10_matricial_leida_de_csv[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top10_matricial_leida_de_csv[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop10)
  lista_metrica_2_top10_leida_de_csv.append(metrica_2_para_par_i)

### Aplico métrica 2 forma original:

In [107]:
print('Calculando métrica 2')

Calculando métrica 2


In [108]:
# Métrica 2 en ranking de 10 elementos
lista_metrica_2_top10 = []
posicion_contranarrativas_encontradas = 8 
posicion_contranarrativas_totales = 10
cantidad_elementos_del_rankingTop10 = 10

for i in range(0, len(lista_pares_métrica_1_top10)):
  metrica_2_para_par_i = lista_pares_métrica_1_top10[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top10[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop10)
  lista_metrica_2_top10.append(metrica_2_para_par_i)

In [109]:
# Métrica 2 en ranking de 10 elementos con método matricial
lista_metrica_2_top10_matricial = []
posicion_contranarrativas_encontradas = 8 
posicion_contranarrativas_totales = 10
cantidad_elementos_del_rankingTop10 = 10

for i in range(0, len(lista_pares_métrica_1_top10_matricial)):
  metrica_2_para_par_i_matricial = lista_pares_métrica_1_top10_matricial[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top10_matricial[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop10)
  lista_metrica_2_top10_matricial.append(metrica_2_para_par_i_matricial)

In [110]:
# Métrica 2 en ranking de 10 elementos random
lista_metrica_2_top10_random = []
posicion_contranarrativas_encontradas = 8 
posicion_contranarrativas_totales = 10
cantidad_elementos_del_rankingTop10 = 10

for i in range(0, len(lista_pares_métrica_1_top10_random)):
  metrica_2_para_par_i = lista_pares_métrica_1_top10_random[i][posicion_contranarrativas_encontradas]/min(lista_pares_métrica_1_top10_random[i][posicion_contranarrativas_totales],cantidad_elementos_del_rankingTop10)
  lista_metrica_2_top10_random.append(metrica_2_para_par_i)

###Sanity check comparando la métrica 2 de la forma original vs leída desde el csv.

In [111]:
lista_metrica_2_top10_matricial == lista_metrica_2_top10_leida_de_csv

True

# Métrica 3: promedio de métrica 2:

metrica_3 será el promedio de la metrica_2.

##Aplico métrica 3 leída desde csv:

In [112]:
metrica_3_top10_csv = sum(lista_metrica_2_top10_leida_de_csv)/len(lista_metrica_2_top10_leida_de_csv)
print('Métrica 3, con ranking de 10 elementos:', metrica_3_top10_csv)

Métrica 3, con ranking de 10 elementos: 0.32426513815402724


##Aplico métrica 3 de forma original:

###Fixme:acá hay muchísimo código repedito, sacar.

In [113]:
metrica_3_top10 = sum(lista_metrica_2_top10)/len(lista_metrica_2_top10)
print('Métrica 3, con ranking de 10 elementos:', metrica_3_top10)

Métrica 3, con ranking de 10 elementos: 0.32426513815402724


En una de las ejecuciones (100 odio y 100 contranarrat), el método matricial me dió:

Métrica 3, con ranking de 10 elementos: 0.14830262258723031

En mi compu (100 odio y 100 contranarrat), el método matricial me dió:
0.14837975427981692 

En mi compu (todos los odio y 100 contranarrat), el método matricial me dió:

0.16168990456515187

En una de las ejecuciones (todos los odio y 100 contranarrat), el método matricial me dió:

Métrica 3, con ranking de 10 elementos: 0.15029037619501082

En una de las ejecuciones (todos los odio y 200 contranarrat), el método matricial me dió: 

Métrica 3, con ranking de 10 elementos: 0.15722574064607484


In [114]:
metrica_3_top10_matricial = sum(lista_metrica_2_top10_matricial)/len(lista_metrica_2_top10_matricial)
print('Métrica 3, con ranking de 10 elementos:', metrica_3_top10_matricial)

Métrica 3, con ranking de 10 elementos: 0.32426513815402724


In [115]:
# Calculo la métrica 3 para el ranking random de 10 elementos.
metrica_3_top10_random = sum(lista_metrica_2_top10_random)/len(lista_metrica_2_top10_random)
print('Métrica 3, con ranking de 10 elementos elegidos al azar:', metrica_3_top10_random)

Métrica 3, con ranking de 10 elementos elegidos al azar: 0.001984126984126984


### Sanity check comparando la métrica 3 de la forma original vs leída desde el csv.

In [116]:
metrica_3_top10 == metrica_3_top10_csv

True

## Guardo los resultados de métrica 3 en archivos (se guardan en la notebook si se corre desde Google Colab y se guardan en la ubicación del script si se corre desde un script .py).

#####Fixme:acá hay muchísimo código repedito, sacar.

In [117]:
print('Guardando outputs de la métrica 3')

Guardando outputs de la métrica 3


In [118]:
# Voy a guardar el output de la métrica 3, top 10 en un archivo:
with open(r'metrica_3_top10_csv.txt', 'w') as fp:
    fp.write(str(metrica_3_top10_csv))
    print('Done')


Done


In [119]:
asdfadsfsa

NameError: ignored

# Sanity checks

## Sanity chequeo que la generación de los embeddings sea determinística

### Chequeo si la generación de los embeddings es determinísitca:

In [ ]:
# Genero los embeddings
embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista1 = model.encode(hate_speech_conan_list_sin_repetidos, convert_to_tensor=True);

In [ ]:
# Genero los embeddings
embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista2 = model.encode(hate_speech_conan_list_sin_repetidos, convert_to_tensor=True);

In [ ]:
torch.equal(embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista1, embeddings_hate_speech_conan_list_sin_repetidos_creado_por_lista2)

### Chequeo si la generación de hate_speech_conan_list_sin_repetidos1 es determinísitca:

In [ ]:
hate_speech_conan_list_sin_repetidos1 =list(set(list(hate_speech_conan.values.flatten()))) #fixme: mega importante: el set está para eliminar repetidos de la lista. Set toma una lista y devuleve un set (creo), posteriormente vuelvo a aplicar list asegurarme que hate_speech_conan_list_sin_repetidos es una lista.

#print(counternarratives_conan_list_sin_repetidos[0:5])

In [ ]:
hate_speech_conan_list_sin_repetidos2 =list(set(list(hate_speech_conan.values.flatten()))) #fixme: mega importante: el set está para eliminar repetidos de la lista. Set toma una lista y devuleve un set (creo), posteriormente vuelvo a aplicar list asegurarme que hate_speech_conan_list_sin_repetidos es una lista.

#print(counternarratives_conan_list_sin_repetidos[0:5])

In [ ]:
hate_speech_conan_list_sin_repetidos2 == hate_speech_conan_list_sin_repetidos1

### Chequeo si la generación de hate_speech_conan es determinísitca:

In [ ]:
hate_speech_conan2 = df1[['hateSpeech']]

In [ ]:
hate_speech_conan1 = df1[['hateSpeech']]

In [ ]:
hate_speech_conan1.equals(hate_speech_conan2)

### Chequeo si la generación de df1 es determinísitca:

In [ ]:
df2 = pd.read_csv(url)

In [ ]:
df3 = pd.read_csv(url)

In [ ]:
df2.equals(df3)

### Chequeos deconectando y reiniciando el tiempo de ejecución: 

#### Chequeo si los embeddings son siempre los mismos cuando me desconecto y vuelvo a ejecutar todo

Me descargo el archivo embeddings_hate_speech.pkl, me desconecto del entorno de ejecución y reinicio el tiempo de ejecución, vuelvo a ejecutar todo. Esto genera un nuevo archivo embeddings_hate_speech.pkl, al comparar ambos archivos de ambas ejecuciones (embeddings_hate_speech.pkl de cada una de las ejecuciones), veo que hay una mínima diferencia de redondeo en aproximadmente 3 líneas de las más de 82000 que tienen los embeddings.

Hago lo mismo para embeddings_counternarratives.pkl y me da que ambos archivos son iugales.

Por lo que ahora puedo concluír que hay determinismo en la generación de embeddings.